# Evaluación 5

```yaml
Equipo: Data Heroes
Integrantes:
    - Paul E. Arizpe Colorado
    - Tania Gúzman Aguirre
    - M. Fernanda Martínez Campos
Grupo: 801
Carrera: LCDN
Sede: Coyoacán
```

### Carga de librerias necesarias

In [14]:
import os
import re
import string
import warnings
import numpy as np
import pandas as pd
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.tag import hmm
from nltk.text import TextCollection
from nltk.stem.porter import PorterStemmer
from tqdm.notebook import tqdm
from datetime import datetime as dt
from wordcloud import WordCloud, ImageColorGenerator
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer, one_hot
import tensorflow as tf

from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import softmax

warnings.filterwarnings('ignore')

# Descargamos los recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# Cargamos las stopwords en español
stop_words = stopwords.words('spanish')

# Para realizar el stemming
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to E:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to E:\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to E:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
print(nltk.__version__)
print(tf.__version__)
print(pd.__version__)

3.9.1
2.19.0
2.2.3


In [3]:
# Carga de documento
def load_doc(pathfile):
    with open(pathfile, 'rt', encoding='utf-8') as file:
        content = file.read()
    return content

In [5]:
# Test Load Document
doc = load_doc('../../tweets/quejas/1.txt')
print(doc)

Reporte 30, día 30
El taller Popocatépetl Motors ( propiedad de 
@LuisMendozaBJ
 ) usa el carril del RTP como estacionamiento y extensión de su patio de maniobras, bloquea banquetas y pasos peatonales. Debido a esto es el causante de varios accidentes viales. 
@UCS_GCDMX
 
@SSC_CDMX


In [10]:
# Limpieza de texto
def clean_text(text):
    # Tokenizar el texto
    tokens = word_tokenize(text)
    
    # Eliminar signos de puntuación y convertir a minúsculas
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    tokens = [re_punc.sub('', w) for w in tokens]
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Eliminar stopwords
    stop_words = set(stopwords.words('spanish'))
    tokens = [w for w in tokens if not w in stop_words]
    
    # Eliminar palabras de un solo carácter
    tokens = [word for word in tokens if len(word) > 1]
    
    return tokens

In [11]:
# test clean_text
tokens = clean_text(doc)
print(tokens[:10])  # Muestra las primeras 10 palabras tokenizadas

['reporte', 'día', 'taller', 'popocatépetl', 'motors', 'propiedad', 'luismendozabj', 'usa', 'carril', 'rtp']


In [12]:
# 
def doc_to_line(filename, vocab):
    doc = load_doc(filename)
    tokens = clean_text(doc)
    tokens = [w for w in tokens if w in vocab]
    return ''.join(tokens)

In [64]:
# Test doc_to_line
filename = '../../tweets/quejas/1.txt'
vocab = Counter({'rtp': 1, 'queja': 2, 'reclamo': 3, 'servicio': 4, 'reporte': 5})
doc2line = doc_to_line(filename, vocab)
print(doc2line)

reportertp


In [59]:
vocab

Counter({('rtp', 1): 1, ('queja', 1): 1})

In [71]:
# Función para procesar los documentos de quejas y sugerencias
def process_docs(directory, vocab, is_train):
    lines = list()
    
    for filename in tqdm(os.listdir(directory)):
        pathfile = rf'{directory}/{filename}'
        texto = doc_to_line(pathfile, vocab)
        print(f'Archivo cargado: {filename}')
        print(texto)
        lines.append(texto)
    return lines

In [73]:
# Test process_docs
directory = '../../tweets/quejas'
is_train = True
lines = process_docs(directory, vocab, is_train)
lines

  0%|          | 0/20 [00:00<?, ?it/s]

Archivo cargado: 1.txt
reportertp
Archivo cargado: 10.txt
rtpservicioservicio
Archivo cargado: 11.txt
servicio
Archivo cargado: 12.txt
serviciortp
Archivo cargado: 14.txt

Archivo cargado: 16.txt
servicio
Archivo cargado: 18.txt

Archivo cargado: 31.txt
serviciortp
Archivo cargado: 32.txt

Archivo cargado: 33.txt
rtp
Archivo cargado: 34.txt
rtpservicio
Archivo cargado: 35.txt
quejartp
Archivo cargado: 36.txt
rtp
Archivo cargado: 37.txt
rtp
Archivo cargado: 38.txt

Archivo cargado: 39.txt
serviciortp
Archivo cargado: 40.txt

Archivo cargado: 5.txt
rtp
Archivo cargado: 7.txt
servicio
Archivo cargado: 8.txt
rtp


['reportertp',
 'rtpservicioservicio',
 'servicio',
 'serviciortp',
 '',
 'servicio',
 '',
 'serviciortp',
 '',
 'rtp',
 'rtpservicio',
 'quejartp',
 'rtp',
 'rtp',
 '',
 'serviciortp',
 '',
 'rtp',
 'servicio',
 'rtp']

In [20]:
def save_list(lines, filname):
    data = '\n'.join(lines)
    with open(filname, 'w') as file:
        file.write(data)

In [21]:
# - Actualizar el vocabulario con las palabras procesadas de cada documento
def update_vocab(vocab, tokens):
    vocab.update(tokens)
    return vocab

In [22]:
# - Procesar los documentos de las quejas y sugerencias y generar un vocabulario.
def process_docs_and_update_vocab(directory, vocab):
    print(f'Procesando documentos en el directorio: {directory}')
    for filename in tqdm(os.listdir(directory)):
        if filename.endswith('.txt'):
            pathfile = f'{directory}/{filename}'
            texto = load_doc(pathfile)
            tokens = clean_text(texto)
            vocab = update_vocab(vocab, tokens)
    return vocab

In [35]:
# - Filtrar las palabras que aparecen con una frecuencia mayor o igual al umbral (5 en este caso) y guardarlas en un archivo.
def save_vocab(lines, filename):
    data = '\n'.join(lines)
    if os.path.exists(filename):
        # Si existe el archivo, agregamos el nuevo vocabulario al final
        with open(filename, 'a+', encoding='utf-8') as file:
            file.write('\n')
            file.write(data)
    else:
        # No existe el archivo, lo creamos
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(data)
    print(f'Vocabulario guardado en: {filename}')

In [74]:
def load_clean_dataset(dir1, dir2, vocab, is_train):
    negative = process_docs(dir1, vocab, is_train)
    positive = process_docs(dir2, vocab, is_train)

    docs = negative + positive #Lista que contiene tanto las quejas como las sugerencias procesadas
    
    labels = [0 for _ in range(len(negative))] + [1 for _ in range(len(positive))]
    return docs, labels

### Preparación de datos 

In [75]:
vocab = Counter()
umbral = 5
tokens = []
directories = ['../../tweets/quejas/', '../../tweets/sugerencias/']
for directory in tqdm(directories):
    vocab = process_docs_and_update_vocab(directory, vocab)
    tokens += [k for k, c in vocab.items() if c >= umbral]

# Guardamos el vocabulario en un archivo
unique_vocab = set(tokens)
save_vocab(unique_vocab, 'vocab.txt')

  0%|          | 0/2 [00:00<?, ?it/s]

Procesando documentos en el directorio: ../../tweets/quejas/


  0%|          | 0/20 [00:00<?, ?it/s]

Procesando documentos en el directorio: ../../tweets/sugerencias/


  0%|          | 0/20 [00:00<?, ?it/s]

Vocabulario guardado en: vocab.txt


In [76]:
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print(vocab)

{'si', 'ciudad', 'ruta', 'hoy', 'servicio', 'día', 'rtpciudaddemex', 'rtp', 'rutas', 'metro', 'mirtp', 'va', 'unidades', 'personas', 'santa'}


In [77]:
# Cargar datos de entrenamiento y prueba
dir1 = r'D:\Workspace\ws_python\computo_cognitivo_labs\tweets\quejas'
dir2 = r'D:\Workspace\ws_python\computo_cognitivo_labs\tweets\sugerencias'
train_docs, y_train = load_clean_dataset(dir1, dir2, vocab, is_train=True)
test_docs, y_test = load_clean_dataset(dir1, dir2, vocab, is_train=False)

  0%|          | 0/20 [00:00<?, ?it/s]

Archivo cargado: 1.txt
díartp
Archivo cargado: 10.txt
rtpciudadserviciometroserviciortpciudaddemex
Archivo cargado: 11.txt
metroserviciounidadesrtpciudaddemexpersonas
Archivo cargado: 12.txt
serviciortpciudaddemexrutasrtpsantametro
Archivo cargado: 14.txt

Archivo cargado: 16.txt
servicio
Archivo cargado: 18.txt

Archivo cargado: 31.txt
serviciorutasantapersonasrtp
Archivo cargado: 32.txt
rutasantava
Archivo cargado: 33.txt
rtp
Archivo cargado: 34.txt
rtpciudadunidadesrutaservicio
Archivo cargado: 35.txt
rtpdíahoyrutasanta
Archivo cargado: 36.txt
rtp
Archivo cargado: 37.txt
rtp
Archivo cargado: 38.txt

Archivo cargado: 39.txt
serviciortpmetrounidadesunidades
Archivo cargado: 40.txt

Archivo cargado: 5.txt
rtpmetro
Archivo cargado: 7.txt
serviciorutartpciudaddemex
Archivo cargado: 8.txt
rtpdía


  0%|          | 0/20 [00:00<?, ?it/s]

Archivo cargado: 13.txt
mirtppersonas
Archivo cargado: 15.txt
mirtpdíadía
Archivo cargado: 17.txt
vahoy
Archivo cargado: 19.txt
mirtprutasantapersonasservicio
Archivo cargado: 2.txt
mirtp
Archivo cargado: 20.txt
rutasvartprutasunidadesrutasi
Archivo cargado: 21.txt
rutasvartprutasunidadesrutasi
Archivo cargado: 22.txt

Archivo cargado: 23.txt
unidadessivartp
Archivo cargado: 24.txt
rtpciudaddemex
Archivo cargado: 25.txt
unidadessi
Archivo cargado: 26.txt

Archivo cargado: 27.txt
serviciohoyrtpciudadsiservicio
Archivo cargado: 28.txt
unidadesrtpciudad
Archivo cargado: 29.txt
hoy
Archivo cargado: 3.txt
rutartpciudaddemexdíaciudadrtpmetrociudad
Archivo cargado: 30.txt
mirtpservicio
Archivo cargado: 4.txt
mirtprutasanta
Archivo cargado: 6.txt
rtp
Archivo cargado: 9.txt
rutartphoyrtpciudaddemexsantapersonas


  0%|          | 0/20 [00:00<?, ?it/s]

Archivo cargado: 1.txt
díartp
Archivo cargado: 10.txt
rtpciudadserviciometroserviciortpciudaddemex
Archivo cargado: 11.txt
metroserviciounidadesrtpciudaddemexpersonas
Archivo cargado: 12.txt
serviciortpciudaddemexrutasrtpsantametro
Archivo cargado: 14.txt

Archivo cargado: 16.txt
servicio
Archivo cargado: 18.txt

Archivo cargado: 31.txt
serviciorutasantapersonasrtp
Archivo cargado: 32.txt
rutasantava
Archivo cargado: 33.txt
rtp
Archivo cargado: 34.txt
rtpciudadunidadesrutaservicio
Archivo cargado: 35.txt
rtpdíahoyrutasanta
Archivo cargado: 36.txt
rtp
Archivo cargado: 37.txt
rtp
Archivo cargado: 38.txt

Archivo cargado: 39.txt
serviciortpmetrounidadesunidades
Archivo cargado: 40.txt

Archivo cargado: 5.txt
rtpmetro
Archivo cargado: 7.txt
serviciorutartpciudaddemex
Archivo cargado: 8.txt
rtpdía


  0%|          | 0/20 [00:00<?, ?it/s]

Archivo cargado: 13.txt
mirtppersonas
Archivo cargado: 15.txt
mirtpdíadía
Archivo cargado: 17.txt
vahoy
Archivo cargado: 19.txt
mirtprutasantapersonasservicio
Archivo cargado: 2.txt
mirtp
Archivo cargado: 20.txt
rutasvartprutasunidadesrutasi
Archivo cargado: 21.txt
rutasvartprutasunidadesrutasi
Archivo cargado: 22.txt

Archivo cargado: 23.txt
unidadessivartp
Archivo cargado: 24.txt
rtpciudaddemex
Archivo cargado: 25.txt
unidadessi
Archivo cargado: 26.txt

Archivo cargado: 27.txt
serviciohoyrtpciudadsiservicio
Archivo cargado: 28.txt
unidadesrtpciudad
Archivo cargado: 29.txt
hoy
Archivo cargado: 3.txt
rutartpciudaddemexdíaciudadrtpmetrociudad
Archivo cargado: 30.txt
mirtpservicio
Archivo cargado: 4.txt
mirtprutasanta
Archivo cargado: 6.txt
rtp
Archivo cargado: 9.txt
rutartphoyrtpciudaddemexsantapersonas


In [78]:
# Crear el tokenizador y entrenarlo con los datos de entrenamiento
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_docs)

In [79]:
train_docs, y_train

(['díartp',
  'rtpciudadserviciometroserviciortpciudaddemex',
  'metroserviciounidadesrtpciudaddemexpersonas',
  'serviciortpciudaddemexrutasrtpsantametro',
  '',
  'servicio',
  '',
  'serviciorutasantapersonasrtp',
  'rutasantava',
  'rtp',
  'rtpciudadunidadesrutaservicio',
  'rtpdíahoyrutasanta',
  'rtp',
  'rtp',
  '',
  'serviciortpmetrounidadesunidades',
  '',
  'rtpmetro',
  'serviciorutartpciudaddemex',
  'rtpdía',
  'mirtppersonas',
  'mirtpdíadía',
  'vahoy',
  'mirtprutasantapersonasservicio',
  'mirtp',
  'rutasvartprutasunidadesrutasi',
  'rutasvartprutasunidadesrutasi',
  '',
  'unidadessivartp',
  'rtpciudaddemex',
  'unidadessi',
  '',
  'serviciohoyrtpciudadsiservicio',
  'unidadesrtpciudad',
  'hoy',
  'rutartpciudaddemexdíaciudadrtpmetrociudad',
  'mirtpservicio',
  'mirtprutasanta',
  'rtp',
  'rutartphoyrtpciudaddemexsantapersonas'],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  

### Creando la Red Neuronal

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import plot_model
import pydot
from tensorflow.keras.layers import Input
from keras.utils import plot_model

In [80]:
# Convertir los documentos en matrices con las `num_words` en modeo binario
X_train = tokenizer.texts_to_matrix(train_docs, mode='binary')
X_test = tokenizer.texts_to_matrix(test_docs, mode='binary')

In [81]:
X_train.shape, X_test.shape

((40, 31), (40, 31))

In [82]:
# Obtener el número de características
n_words = X_test.shape[1]  # Número de características (palabras) que se usarán
n_words

31

In [83]:
# Convertir a float32 para compatibilidad
X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [43]:
#Modelo de analisis de reseñas

def define_model(n_words):
    model = Sequential()
    model.add(Input(shape=(n_words,)))  # Usamos Input en lugar de input_shape
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [88]:
def predict_sentiment(review, vocad, tokenizer, model):
    # Preprocesar el texto de la reseña
    tokens = clean_text(review)
    tokens = [w for w in tokens if w in vocab] # Filtrar palabras según el vocabulario
    lines = ''.join(tokens)
    
    # Convertir la reseña a una matriz
    encoded = tokenizer.texts_to_matrix([review], mode='binary')
    
    # Realizar la predicción
    y_pred = model.predict(encoded, verbose=0)
    porcentaje_pos = y_pred[0][0]
    
    if round(porcentaje_pos) == 0:
        return (1 - porcentaje_pos), 'Negativa'
    return porcentaje_pos, 'Positiva'

In [84]:
# Crear el tokenizador y entrenarlo con los datos de entrenamiento
tokenizer = Tokenizer(num_words=num_words)  # Limitar el tokenizador a las `num_words` más frecuentes
tokenizer.fit_on_texts(train_docs)  # Entrenar el tokenizador con los documentos de entrenamiento

# Convertir los documentos en matrices con las `num_words` más frecuentes
X_train = tokenizer.texts_to_matrix(train_docs, mode='binary')
X_test = tokenizer.texts_to_matrix(test_docs, mode='binary')

# Obtener el número de características
n_words = X_test.shape[1]  # Número de características (palabras) que se usarán

# Definir el modelo
model = define_model(n_words)

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, verbose=2)

# Evaluar el modelo
loss, acc = model.evaluate(X_test, y_test, verbose=0)

# Imprimir la precisión en el conjunto de prueba
print('Accuracy test: %f' % (acc * 100))

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 50)             │       250,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 250,101 (976.96 KB)

 Trainable params: 250,101 (976.96 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
2/2 - 1s - 366ms/step - accuracy: 0.5250 - loss: 0.6938
Epoch 2/10
2/2 - 0s - 32ms/step - accuracy: 0.5250 - loss: 0.6901
Epoch 3/10
2/2 - 0s - 28ms/step - accuracy: 0.6000 - loss: 0.6871
Epoch 4/10
2/2 - 0s - 26ms/step - accuracy: 0.6000 - loss: 0.6842
Epoch 5/10
2/2 - 0s - 28ms/step - accuracy: 0.6750 - loss: 0.6812
Epoch 6/10
2/2 - 0s - 29ms/step - accuracy: 0.7500 - loss: 0.6785
Epoch 7/10
2/2 - 0s - 32ms/step - accuracy: 0.8250 - loss: 0.6758
Epoch 8/10
2/2 - 0s - 42ms/step - accuracy: 0.8250 - loss: 0.6730
Epoch 9/10
2/2 - 0s - 27ms/step - accuracy: 0.8500 - loss: 0.6703
Epoch 10/10
2/2 - 0s - 25ms/step - accuracy: 0.9000 - loss: 0.6674
Accuracy test: 92.500001


In [ ]:
txt_queja = 'No me gusta el servicio de atención al cliente. No me han resuelto el problema.'

perc_queja, review = predict_sentiment(txt_queja, vocab, tokenizer, model)
print('Review [%s]\nSentiment: %s (%.3f%%):' % (txt_queja, review, perc_queja * 100))



Review [No me gusta el servicio de atención al cliente. No me han resuelto el problema.]
Sentiment: Negativa (52.800%):
Review [Me gustaría que mejoraran el servicio de atención al cliente. Me gustaría que me llamen para resolver el problema.]
Sentiment: Negativa (52.800%):


In [ ]:
txt_sugerencia = 'Me gustaría que mejoraran el servicio de atención al cliente. Me gustaría que me llamen para resolver el problema.'

perc_sugerencia, review2 = predict_sentiment(txt_sugerencia, vocab, tokenizer, model)
print('Review [%s]\nSentiment: %s (%.3f%%):' % (txt_sugerencia, review2, perc_sugerencia * 100))


Me gustaría que mejoraran el servicio de atención al cliente. Me gustaría que me llamen para resolver el problema.
Sentiment: Positiva (72.800%)
